# Build SQLite Database for E-Commerce Project

This notebook creates a local SQLite database for the e-commerce project and loads the generated CSV files into tables.

The steps are:
1. Connect to the database  
2. Create the tables (users, products, sessions, orders, order_items, marketing_spend)  
3. Import data from `data/raw/` into the tables  
4. Do a few quick checks to make sure everything loaded correctly  


In [19]:
import sqlite3
import pandas as pd

# Connect to (or create) the SQLite database file
conn = sqlite3.connect('../data/processed/ecommerce.db')
cursor = conn.cursor()

# Create the main tables if they don't already exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY,
    signup_date TEXT,
    email TEXT,
    country TEXT,
    marketing_channel TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    name TEXT,
    category TEXT,
    price REAL,
    cost REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS sessions (
    session_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    session_date TEXT,
    source TEXT,
    pages_viewed INTEGER,
    duration_sec INTEGER
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    order_date TEXT,
    total_amount REAL,
    order_status TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    price REAL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS marketing_spend (
    channel TEXT,
    spend_date TEXT,
    spend_amount REAL
)
''')

conn.commit()
print("Database and tables created.")


Database and tables created.


## Load data from CSV files into the database

The CSV files generated in `00_generate_data.ipynb` are stored in `data/raw/`.  
Here we load each file into its matching table in the SQLite database.


In [20]:
# List of tables to load
tables = ['users', 'products', 'sessions', 'orders', 'order_items', 'marketing_spend']

for table in tables:
    df = pd.read_csv(f'../data/raw/{table}.csv')
    df.to_sql(table, conn, if_exists='replace', index=False)

print("All CSV files have been loaded into the database.")


All CSV files have been loaded into the database.


## Quick row counts per table

Before moving on, it's useful to check how many records we have in each table.


In [21]:
for table in tables:
    count = pd.read_sql(f"SELECT COUNT(*) AS rows FROM {table}", conn)
    print(f"{table}: {count['rows'][0]} rows")


users: 500 rows
products: 50 rows
sessions: 1494 rows
orders: 1037 rows
order_items: 3200 rows
marketing_spend: 48 rows


### Close the database connection

The database is now ready to be used in the analysis notebook.


In [22]:
conn.close()
print("Database connection closed.")


Database connection closed.
